On peut faire les fonctions impératives pour ce projet

In [16]:
import math
import random as rd
import sys
def algo_Euclide(a,b):
    if (b==0):
        return(a)
    else:
        return algo_Euclide(b, a%b)


def gen_pochon(n):
    tab=[]
    b=rd.randint(0,sys.maxsize//n)
    tab.append(b)
    n=n-1
    somme=b
    while (n>0):
        b=rd.randint(somme, somme + sys.maxsize//n)
        n=n-1
        somme+=b
        tab.append(b)
    return tab

print(gen_pochon(100))

        
    

[19801452568225929, 68440454761984322, 114619056751660450, 286447585122724099, 491223862786110486, 1018471091703333541, 2022776930952307625, 4116797237667106562, 8160554061643822405, 16361969758680857678, 32760315563320168246, 65448853886519996831, 130882439343872299412, 261753638895151548431, 523550950234074340017, 1047079483874044709151, 2094139282128584535512, 4188284570632069749014, 8376585947282646339545, 16753204052672567033016, 33506416610638804434241, 67012802047031759991383, 134025593031527471284827, 268051259052134305278014, 536102537014812963807940, 1072205065957190523975160, 2144410132603888114666535, 4288820235683724672565863, 8577640511082836119126842, 17155280923458984894208582, 34310561894216849823849534, 68621123851493004735243267, 137242247622872416654812333, 274484495310768828568806926, 548968990623341777216187714, 1097937981261204567522655648, 2195875962473175913941908410, 4391751925006978445708463982, 8783503849948153889774735542, 17567007699907867413394084677, 351